# Visualizing Damage Buildings and Infrastructure in Gaza

This notebook provides a visual analysis of Synthetic Aperture Radar (SAR) data to assess and visualize potential damage in the Gaza Strip. SAR data, known for its ability to capture ground features and changes regardless of weather conditions or time of day, can be instrumental in identifying changes in infrastructure or land features following conflict or disasters.

You can now interact with this map by selecting/deselecting different damage layers to visualize where buildings are most impacted in the Gaza Strip. Upon visual inspection, you can see:

1. **Damage:** You can select and deselect the layers to see damages to roads, buildings and points of interest. The damage value ranges from $[0,1]$, with $1$ indicating higher level of damage.
2. **Conflict:** You can select this layer to view areas with reported events and fatalities.

<iframe width="100%" height="500px" src="https://studio.foursquare.com/public/26ba3dfb-0770-4cc6-b6fa-2ee2878580fc/embed" frameborder="0" allowfullscreen></iframe>

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Damage assessment in the Gaza Strip utilizing SAR imagery to evaluate the extent of destruction The damage map above is based on preliminary estimates and have not been verified through field survey or satellite imagery. It does not reflect the final estimations by the World Bank. Country borders or names do not necessarily reflect the World Bank Group's official position. This map is for illustrative purposes and does not imply the expression of any opinion on the part of the World Bank, concerning the legal status of any country or territory or concerning the delimitation of frontiers or boundaries.
```

```{note}
The results shown in this analysis only reflect the damages until **April 10, 2024**. As the team completes further analysis, the notebook will be updated. 
```